In [24]:
TABLE1_PATH = "../data/public/main1.csv"
TABLE2_PATH = "../data/public/main2.csv"
TABLE3_PATH = "../data/public/main3.csv"

In [42]:
import pandas as pd

LIMIT_SIZE = 10000
table1 = pd.read_csv(TABLE1_PATH, nrows=LIMIT_SIZE)
table2 = pd.read_csv(TABLE2_PATH, nrows=LIMIT_SIZE)
table3 = pd.read_csv(TABLE3_PATH, nrows=LIMIT_SIZE)

table1.to_csv("../data/subset_main1.csv", index=False)
table2.to_csv("../data/subset_main2.csv", index=False)
table3.to_csv("../data/subset_main3.csv", index=False)

In [31]:
print(f"Table 1 columns: {list(table1.columns)}")
print(f"Table 2 columns: {list(table2.columns)}")
print(f"Table 3 columns: {list(table3.columns)}")

Table 1 columns: ['uid', 'full_name', 'email', 'address', 'sex', 'birthdate', 'phone']
Table 2 columns: ['uid', 'first_name', 'middle_name', 'last_name', 'birthdate', 'phone', 'address']
Table 3 columns: ['uid', 'name', 'email', 'birthdate', 'sex']


In [32]:
total_columns = ["uid", "name", "birthdate", "phone", "address", "sex", "email"]

In [36]:
import mapply

mapply.init(
    n_workers=-1,
    chunk_size=100,
    max_chunks_per_worker=8,
    progressbar=False,
)

In [ ]:
import recordlinkage

indexer = recordlinkage.Index()
indexer.block("surname")
candidate_links = indexer.index(table1, table2)

In [ ]:
c = recordlinkage.Compare()

c.string("name_a", "name_b", method="jarowinkler", threshold=0.85)
c.exact("sex", "gender")
c.date("dob", "date_of_birth")
c.string("str_name", "streetname", method="damerau_levenshtein", threshold=0.7)
c.exact("place", "placename")
c.numeric("income", "income", method="gauss", offset=3, scale=3, missing_value=0.5)

# The comparison vectors
feature_vectors = c.compute(candidate_links, df_a, df_b)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import MiniBatchKMeans
import time

# Генерация синтетических данных
n_samples = 40000000  # 1 миллион записей
n_features = 50  # Две признака
n_clusters = 100  # Количество кластеров

# Генерация данных с помощью make_blobs
X, _ = make_blobs(n_samples=n_samples, centers=n_clusters, cluster_std=0.60, random_state=0)

# Применение MiniBatch K-means
batch_size = 10000  # Размер батча
kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size)

# Подсчет времени работы
start_time = time.time()

# Обучение модели
kmeans.fit(X)

# Получение предсказаний кластеров
labels = kmeans.predict(X)

# Подсчет времени выполнения
end_time = time.time()
execution_time = end_time - start_time
print(f"Время работы MiniBatch K-means: {execution_time:.2f} секунд")